In [3]:
import numpy as np
import pandas as pd
import sympy as sp
from sympy import *
import math
import os

In [4]:
#Input Pressure and Temperature values
P = float(input("Enter pressure in psia:")) # psia
T = float(input("Enter Temperature in Farenheit: ")) # Farenheit

In [5]:
#Import Components Properties
component_properties = pd.read_excel("components.xlsx")
component_properties

,component,composition,Pci,Tci,wi,Mwi,Vci
0,N2,0.0200,493.1,227.49,0.040,28.013,0.0510
1,C1,0.8866,666.4,343.33,0.008,16.043,0.0988
2,C2,0.0492,706.5,549.92,0.098,30.070,0.0783
3,C3,0.0246,616.0,666.06,0.152,44.097,0.0727
4,nC4,0.0098,550.6,765.62,0.193,58.123,0.0703
5,nC5,0.0098,488.6,845.80,0.251,72.151,0.0675


In [6]:
#Import binary coefficients
d= pd.read_excel("binary.xlsx").to_numpy()
d

array([[0.    , 0.018 , 0.039 , 0.046 , 0.047 , 0.048 ],
       [0.018 , 0.    , 0.005 , 0.01  , 0.0145, 0.0182],
       [0.039 , 0.005 , 0.    , 0.0017, 0.0032, 0.0048],
       [0.046 , 0.01  , 0.0017, 0.    , 0.0012, 0.0024],
       [0.047 , 0.0145, 0.0032, 0.0012, 0.    , 0.0008],
       [0.048 , 0.0182, 0.0048, 0.0024, 0.0008, 0.    ]])

In [7]:
Pci = component_properties['Pci'] 
Tci = component_properties['Tci']

Pri= P/Pci
Tri= (T+459.67 )/Tci

print('Reduced Pressure,  Pri:\n', Pri)
print('Reduced Temperature, Tri:\n',Tri)

Reduced Pressure,  Pri:
 0    2.027986
1    1.500600
2    1.415428
3    1.623377
4    1.816201
5    2.046664
Name: Pci, dtype: float64
Reduced Temperature, Tri:
 0    1.778847
1    1.178662
2    0.735871
3    0.607558
4    0.528552
5    0.478446
Name: Tci, dtype: float64


In [8]:
wi = component_properties['wi'] #accentric factor

w_a = 0.45723553
w_b = 0.07779607

for i in wi:
    if i<0.49:
        mi = 0.3796+1.54226*wi-0.2699*wi**2
    else:
        mi = 0.379642+1.48503*wi-0.164423*wi**2+0.016667*wi**3
        
Ai =(w_a*(1+mi*(1-Tri**0.5))**2*(Pri/Tri**2)).replace(np.nan,0).to_numpy()
Bi = ((w_b*Pri)/Tri).to_numpy()
print('Ai:',Ai)
print('Bi:',Bi)

Ai: [0.21315405 0.46128023 1.38137769 2.58607939 4.15396658 6.19628096]
Bi: [0.0886919  0.0990452  0.14963872 0.20786879 0.26732139 0.33279046]


In [9]:
#Calculating A and B

c = component_properties['composition'].to_numpy()
n= len(c)

B =  np.sum(c * Bi)

A=0
for i in range (n):
    for j in range (n):
        A += (1-d[i,j])*(Ai[i]*Ai[j])**0.5*c[i]*c[j] 
        
print('A :', A)
print('B :', B)

A : 0.5651691201327123
B : 0.107944202075142


In [10]:
#Calculating Z 

m1 = 1+2**0.5
m2 = 1-2**0.5
p=1
q=(m1+m2-1)*B-1
r=(A+m1*m2*B**2-(m1+m2)*B*(B+1))
s=-A*B-m1*m2*B**2*(B+1)

Z = np.roots([p, q, r, s])

print('For Pressure:',P , 'and', 'Temperature:',T )
print('                          A:', round(A,4))
print('                          B:', round(B,4))
print('                          Z:', Z)
for i in range(len(Z)):
    if np.isreal(Z[i]):
        print('Without imaginery values, Z:', np.real(Z[i]))
        
print('                      Z_max:', max(Z))
print('                      Z_min:', min(Z))

For Pressure: 1000.0 and Temperature: -55.0
                          A: 0.5652
                          B: 0.1079
                          Z: [0.41292058+0.j         0.23956761+0.24307906j 0.23956761-0.24307906j]
Without imaginery values, Z: 0.4129205765436229
                      Z_max: (0.4129205765436229+0j)
                      Z_min: (0.23956761069061733-0.24307906481308494j)


In [22]:

Ki = (1/Pri)*((5.37*(1+wi)*(1-1/Tri)).apply(math.exp))

Ki 

0    5.687073
1    1.513815
2    0.085111
3    0.011329
4    0.001816
5    0.000323
dtype: float64

In [43]:
g_V = lambda v:sum(c*(Ki - 1)/(v*(Ki - 1)+1))
dg_V = lambda v:-sum(c*((Ki-1)**2)/((v*(Ki-1) + 1)**2))

A = sum(c*(Ki-1))
B = sum(c*(Ki-1)/Ki)

v = A/(A - B) 

while abs(g_V(v))>0.0001:
    v = v - g_V(v)/dg_V(v)

print(f"Composition in vapor phase is: {v}")
l = 1 - v
print(f"Composition in liquid phase is: {l}")



Composition in vapor phase is: 0.7754182515439129
Composition in liquid phase is: 0.22458174845608714


In [42]:
Xi = c/(1 + v*(Ki-1))
Yi = Xi*Ki

def Aa(c):
    return [sum(c[i]*c*(Ai[i]*Ai)**0.5) for i in range(len(c))]

Al = sum(Aa(Xi))
Av = sum(Aa(Yi))
Bl = sum(Xi*Bi)
Bv = sum(Yi*Bi)

print(f"A in Vapour phase {Av} and liquid phase {Al}")
print(f"B in Vapour phase {Bv} and liquid phase {Bl}")



A in Vapour phase 0.4654376987788452 and liquid phase 0.9959953217004294
B in Vapour phase 0.0996666779058611 and liquid phase 0.13652418848357134
